<a href="https://colab.research.google.com/github/Fernsrea/reimagined-eureka/blob/main/CIFAR_10_CIFAR_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/facebookresearch/flow_matching.git
%cd flow_matching

fatal: destination path 'flow_matching' already exists and is not an empty directory.
/content/flow_matching


In [2]:
%cd examples/image

/content/flow_matching/examples/image


In [3]:
!pip install -r requirements.txt

In [4]:
!pip install torchdiffeq

In [5]:
!pip install flow_matching

In [ ]:
!python train.py --dataset cifar10 --output_dir output_cifar10 --batch_size 64 --accum_iter=1 --eval_frequency=100 --epochs=3000 --class_drop_prob=1.0 --cfg_scale=0.0 --compute_fid --ode_method heun2 --ode_options '{"nfe": 50}' --use_ema --edm_schedule --skewed_timesteps

Not using distributed mode
2025-04-17 08:53:07 INFO     job dir: /content/flow_matching/examples/image
2025-04-17 08:53:07 INFO     Namespace(batch_size=64,
epochs=3000,
accum_iter=1,
lr=0.0001,
optimizer_betas=[0.9,
0.95],
decay_lr=False,
class_drop_prob=1.0,
skewed_timesteps=True,
edm_schedule=True,
use_ema=True,
dataset='cifar10',
data_path='./data/image_generation',
output_dir='output_cifar10',
ode_method='heun2',
ode_options={'nfe': 50},
sym=0.0,
temp=1.0,
sym_func=False,
sampling_dtype='float32',
cfg_scale=0.0,
fid_samples=50000,
device='cuda',
seed=0,
resume='',
start_epoch=0,
eval_only=False,
eval_frequency=100,
compute_fid=True,
save_fid_samples=False,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://',
test_run=False,
discrete_flow_matching=False,
discrete_fm_steps=1024,
distributed=False)
2025-04-17 08:53:07 INFO     Saving args to output_cifar10/args.json
2025-04-17 08:53:07 INFO     Initializing Dataset: cifar10
100% 170M/170M [

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir output_cifar10


In [6]:
!pip install submitit

In [9]:
import os
from pathlib import Path
import uuid  # Import the uuid module (although it's already in your submitit_train.py)

shared_dir = "/content/shared_experiments" # Choose a path within /content/

if not Path(shared_dir).is_dir():
    os.makedirs(shared_dir, exist_ok=True)

def get_shared_folder(shared_dir: str) -> Path:
    user = os.getenv("USER", "default_user") # Colab might not always have USER set
    p = Path(shared_dir) / user / "experiments"
    p.mkdir(parents=True, exist_ok=True)
    return p

def get_init_file(shared_dir: str):
    shared_folder = get_shared_folder(shared_dir)
    init_file = shared_folder / f"{uuid.uuid4().hex}_init"
    if init_file.exists():
        os.remove(str(init_file))
    return init_file

# You can optionally run these lines to see the paths that will be used
job_dir = get_shared_folder(shared_dir) / "my_job" # Example job directory
print(f"Job directory: {job_dir}")
init_file = get_init_file(shared_dir)
print(f"Init file: {init_file}")

Job directory: /content/shared_experiments/default_user/experiments/my_job
Init file: /content/shared_experiments/default_user/experiments/91b8b8755bda4b568b191291ed315ac4_init


In [11]:
!python submitit_train.py \
    --dataset=cifar10 \
    --batch_size=64 \
    --nodes=1 \
    --accum_iter=1 \
    --eval_frequency=100 \
    --epochs=3000 \
    --class_drop_prob=1.0 \
    --cfg_scale=0.0 \
    --compute_fid \
    --ode_method heun2 \
    --ode_options '{"nfe": 50}' \
    --use_ema \
    --edm_schedule \
    --skewed_timesteps \
    --output_dir output_cifar10_submitit \
    --shared_dir "/content/shared_experiments"

2025-04-21 18:00:47 INFO     Submitted job 15124
